In [1]:
from pathlib import Path
import numpy as np
import tifffile
from skimage.exposure import match_histograms, rescale_intensity
from skimage.restoration import calibrate_denoiser, denoise_tv_chambolle
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import cv2 as cv



In [ ]:
fname_SI = '020425\\Analysis_022025\\HiLo_Python\\first_9999.tif'
#fname_UI = '020425\\Analysis_022025\\HiLo_Python\\first_9999_pWF.tif'
    
input_file_path_SI = Path(fname_SI)
#input_file_path_UI = Path(fname_UI)
root_path = input_file_path_SI.parents[0]
img_SI = tifffile.imread(input_file_path_SI)
img_SI = img_SI.astype(np.float32)
#img_UI = tifffile.imread(input_file_path_UI)
#img_UI = img_UI.astype(np.float32)
nt = img_SI.shape[0]
ny = img_SI.shape[1]
nx = img_SI.shape[2]


In [28]:
ks = 101
kf = 17

img_SI_LA = np.zeros((nt-2,ny,nx),dtype=np.float32)
img_UI_LA = np.zeros((nt-2,ny,nx),dtype=np.float32)
CP = np.zeros((nt-2,ny,nx),dtype=np.float32)
I_Lo = np.zeros((nt-2,ny,nx),dtype=np.float32)
OS = np.zeros((nt-2,ny,nx),dtype=np.float32)
R_Lo = np.zeros((nt-2,ny,nx),dtype=np.float32)
WF = np.zeros((nt-2,ny,nx),dtype=np.float32)


for i in range(nt-2): 
    
    im1 = img_SI[i,:,:]
    im2 = img_SI[i+1,:,:]
    im3 = img_SI[i+2,:,:]
    
    OS[i,:,:] = (3/np.sqrt(2))*np.sqrt((im1-im2)**2 + (im2-im3)**2 + (im3-im1)**2)
    WF[i,:,:] = (1/3)*(im1 + im2 + im3)
    R_Lo[i,:,:] = cv.GaussianBlur(OS[i,:,:], (kf,kf), 0)
    
    img_SI_LA[i,:,:] = cv.GaussianBlur(img_SI[i,:,:], (ks,ks), 0)
    img_UI_LA[i,:,:] = cv.GaussianBlur(WF[i,:,:], (ks,ks), 0)



    CP[i,:,:] = cv.GaussianBlur(np.absolute(img_SI[i,:,:] - WF[i,:,:]), (ks,ks), 0) 
    I_Lo[i,:,:] = cv.GaussianBlur(CP[i,:,:]*WF[i,:,:], (kf,kf), 0) 

I_Lo = rescale_intensity(I_Lo,out_range=(0,65535)).astype(np.uint16)
R_Lo = rescale_intensity(R_Lo,out_range=(0,65535)).astype(np.uint16)
OS = rescale_intensity(OS,out_range=(0,65535)).astype(np.uint16)
WF = rescale_intensity(WF,out_range=(0,65535)).astype(np.uint16)

tifffile.imsave(fname_SI[:-4]+'_WF.tif', WF[:,:,:])
tifffile.imsave(fname_SI[:-4]+'_R_Lo.tif', R_Lo[:,:,:])
tifffile.imsave(fname_SI[:-4]+'_Lo.tif', I_Lo[:,:,:])
tifffile.imsave(fname_SI[:-4]+'_OS.tif', OS[:,:,:])



